(getting-started)=
# Getting started
## Installation 
The simplest approach is to enter
```bash
pip install xapres
```
in the terminal.

Alternatively, you could clone the repository from GitHub, https://github.com/ldeo-glaciology/xapres, install dependencies using 

```bash
conda env create -f environment.yml
```

and add the xapres repository to your python path before loading the package. If `dir_containing_xapres_repo` is the absolute path to the directory containing the xapres repository, the following will add the repo to your path 

```python
import sys
sys.path.append(dir_containing_xapres_repo + "/xapres/")
```

You can also do this with a relative path as follows:


In [ ]:
import sys
sys.path.append("../../../xapres") # note that this notebook is running in the docs/src directory, so ../../../xapres/ is the correct relative path to use. 

## Loading the package
To load the package into a python session use 
```python
import xapres
```
In this documentation we use

In [2]:
import xapres as xa

## Loading ApRES data into an xarray
One main purpose of the xapres package is to load ApRES data stored in binary files produced by the radar into xarray datasets. This allows for easy manipulation, processing and plotting. 

We will load example data collected on Thwaites glacier as part of the International Thwaites Glacier Collaboration (ITGC). The data were collected in unattended mode, meaning that repeat measurements were taken at the same location (see {ref}`section:apres_survey_structure`).

In [ ]:
data = xa.load.generate_xarray(directory='../data/thwaites/');

100%|██████████| 1/1 [00:00<00:00,  2.50it/s]


The function `xa.load.generate_xarray` finds all the dat files in the directory supplied to it and loads them into an xarray. It also performs a fast fourier transform on the data to compute complex profiles (see [here](https://ldeo-glaciology.github.io/glaciology-intro-book/sections/radar/apres/theory_1.html) for an explanation of the theory).

The following displays the structure of the dataset. 

In [ ]:
data

The two most important data variables are:
:`chirp`: the 'deramped' chirp. I.e. the signal recorded by the radar after the received signal is mixed with the transmitted signal and low-pass filtered. This is an array of real values. 
:`profile`: the profile resulting from an fast- fourier transform of the chirp. This is an array of complex values.

See {ref}`page:how-to-loading` for more details on the structure of this xarray dataset. 


## Plotting
We can now use xarray's powerful, high-level slicing and plotting methods to take a quick look at the data. For example, to plot the 10th chirp from the second burst:

In [ ]:
data.chirp.isel(time=1, chirp_num=9).plot();

To plot the profile from the same chirp, we first make use of the `dB` method that xapres adds to xarray datasets to compute the power in decibels, then use xarray's plot method, specifying the x axis limits: 


In [ ]:
data.profile.isel(time=1, chirp_num=9).dB().plot(xlim=[0, 2500]);

## Stacking
Averaging all the chirps in each burst is generally referred to as stacking in ApRES processing. 

In xarray this is is a straigthforward operation that is easily applied to the whole dataset, i.e. all bursts and all attenuator settings:

In [ ]:
stacked_profiles = data.profile.mean(dim='chirp_num')

To briefly explore the impact of stacking we can plot one profile on top of the stacked profile:

In [ ]:
import matplotlib.pyplot as plt
data.profile.isel(time=1, chirp_num=9).dB().plot(xlim=[0, 2500], label = 'individual profile', add_legend=True)
stacked_profiles.isel(time=1).dB().plot(xlim=[0, 2500], label = 'stacked profile', add_legend=True)
plt.legend();

The bed is more clearly seen around 2200 m in the stacked profile. See {ref}`page:how-to-stacking` for more details on stacking.

## Compute vertical velocity
A common processing step in ApRES data is to use two profiles to compute the vertical displacement of the ice during the time between the two measurements. xapres has a function in the utils module to do this called `displacement_timeseries`:

In [ ]:
d = stacked_profiles.displacement_timeseries()
d

This produced a new xarray dataset with several new variables, including the displacement, the velocity, and their associated uncertainties, as well as the coherence between the two profiles. It also stores the two profiles themselves and computes the strain rate and related quantities. 

We can plot the velocity as a function of depth as follows:

In [ ]:
d.velocity.plot(y = 'bin_depth', yincrease=False, xlim = (-2, 7), ylim = (1200,0));

## Summary
This page has shown you how to get started with the xapres package by loading ApRES data collected, performing some basic processing steps, and plotting the results.

The following pages get it to more detail on how to load ApRES data collected in both unattended and attended mode, save it to a cloud format (zarr), perform the fundamental fft processing step while varying some processing options, and perform the displacement calculation while varying other options.